In [1]:
import qkit
qkit.cfg['load_visa'] = True
qkit.cfg['datadir']=r"C:\Users\weideslab-admin\Documents\Python Scripts\QKIT Implementation"
qkit.start()

from qkit import visa
import time
import h5py
import pandas as pd
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

from qkit.measure.transport import transport
import qkit.measure.samples_class as sc

QKIT configuration initialized -> available as qkit.cfg[...]
Starting QKIT framework ... -> qkit.core.startup
Loading module ... S10_logging.py
Loading module ... S14_setup_directories.py
Loading module ... S16_available_modules.py
Loading module ... S20_check_for_updates.py
Loading module ... S25_info_service.py
Loading module ... S30_qkit_start.py
Loading module ... S65_load_RI_service.py
Loading module ... S70_load_visa.py
Loading module ... S80_load_file_service.py
Loading module ... S85_init_measurement.py


Loading module ... S98_started.py
Loading module ... S99_init_user.py
Initialized the file info database (qkit.fid) in 0.060 seconds.


In [ ]:
keithley_2450_VBS=qkit.instruments.create("keithley_2450_VBS","Keithley_2450_gla", address="TCPIP0::192.168.1.8::inst0::INSTR")
keithley_2450_VDS=qkit.instruments.create("keithley_2450_VDS","Keithley_2450_gla", address="TCPIP0::192.168.1.39::inst0::INSTR")
keithley_2450_VGS=qkit.instruments.create("keithley_2450_VGS","Keithley_2450_gla", address="TCPIP0::192.168.1.50::inst0::INSTR")

In [ ]:
#Keithley settings
keithley_2450_VDS.reset()
keithley_2450_VDS.set_rear()

keithley_2450_VDS._visainstrument.write("SOUR:FUNC VOLT")
keithley_2450_VDS._visainstrument.write("SOUR:VOLT:ILIM "+str(3e-3))
keithley_2450_VDS._visainstrument.write("SENS:CURR:RANG:AUTO ON")
keithley_2450_VDS._visainstrument.write("SENS:CURR:NPLCycles 1")
keithley_2450_VDS._visainstrument.write(":OUTP:STAT ON")
#SMU1.set_nplc(1) # number of power line cycles
#keithley_2450_VDS.meas_delay=0.5 #delay between each measurement in a sweep [seconds]
#keithley_2450_VDS.set_sweep_mode("VOLT","CURR") # set function: bias first, sense second [VV,IV,VI,II]
keithley_2450_VDS.set_4W(0) #set 4point sensing (1) or 2point (0)

keithley_2450_VGS.reset()
keithley_2450_VGS.set_rear()

keithley_2450_VGS._visainstrument.write("SOUR:FUNC VOLT")
keithley_2450_VGS._visainstrument.write("SOUR:VOLT:ILIM "+str(3e-3))
keithley_2450_VGS._visainstrument.write(":OUTP:STAT ON")
#SMU2.set_nplc(1) # number of power line cycles
#keithley_2450_VGS.meas_delay=0.5 #delay between each measurement in a sweep [seconds]
#keithley_2450_VGS.set_sweep_mode("VOLT","CURR") # set function: bias first, sense second [VV,IV,VI,II]
#keithley_2450_VGS.set_4W(0) #set 4point sensing (1) or 2point (0)

keithley_2450_VBS.reset()
keithley_2450_VBS.set_rear()

keithley_2450_VBS._visainstrument.write("SOUR:FUNC VOLT")
keithley_2450_VBS._visainstrument.write("SOUR:VOLT:ILIM "+str(3e-3))
keithley_2450_VBS._visainstrument.write(":OUTP:STAT ON")
#SMU3.set_nplc(1) # number of power line cycles
#keithley_2450_VBS.meas_delay=0.5 #delay between each measurement in a sweep [seconds]
#keithley_2450_VBS.set_sweep_mode("VOLT","CURR") # set function: bias first, sense second [VV,IV,VI,II]
#keithley_2450_VBS.set_4W(0) #set 4point sensing (1) or 2point (0)


In [ ]:
#Initialize transport measurement class with corresponding SMU unit
tr1=transport.transport(keithley_2450_VGS)
sample = sc.Sample()
sample.name = 'Vg'
sample.comment = 'comments'


tr2=transport.transport(keithley_2450_VBS)
sample = sc.Sample()
sample.name = 'Vb'
sample.comment = 'comments'

tr3=transport.transport(keithley_2450_VDS)
sample = sc.Sample()
sample.name = 'Vd'
sample.comment = 'comments'

In [ ]:
step_VBS = 0.3 #[0.5]
step_VGS = 0.3#[0.05]
step_VDS = 0.3#[0.025]


#value_VBS = [round(b, 3) for b in list(np.arange(-0.3,0.7,step_VBS))]  #-0.5 0 0.05 1 1.35 (Body values)
value_VBS = [0,0.1] #for discrete body bias please enter the values here and comment above
value_VGS = [0.2,0.4,0.6] 
#value_VGS = [0,0.5,1]
value_VDS = [round(d, 3) for d in list(np.arange(0,1.21,step_VDS))]
#value_VBS_nmos = [0]   #0 0.05 0.1
#value_VDS_nmos = [round(d, 3) for d in list(np.arange(0,1.9,0.05))]   #0 0.1 ... 1.8     #0 0.9 1.8
#value_VGS_nmos = [round(g, 3) for g in list(np.arange(0.9,1.9,0.9))] 
tr3.measure_transistor_IDS_vs_VDS(keithley_2450_VGS,keithley_2450_VDS,value_VBS,value_VGS,value_VDS,"try1")

In [ ]:
step_VBS = 0.3 #[0.5]
step_VDS = 0.5 #[0.9]
step_VGS = 0.1 #[0.05]


#value_VBS = [round(b, 3) for b in list(np.arange(-0.3,0.7,step_VBS))]  #0 0.05 0.1
value_VBS = [0,0.1] #For discrete body bias please specify values here and comment above
#value_VDS = [0.05] + [round(d, 3) for d in list(np.arange(0.5,1.5,step_VDS))]    #0.05 0.3 ... 1.8
value_VDS = [0.05,0.9,1.8]
value_VGS = [round(g, 3) for g in list(np.arange(0,1.81,step_VGS))]  
tr2.measure_transistor_IDS_vs_VGS(keithley_2450_VGS,keithley_2450_VDS,value_VBS,value_VGS,value_VDS,"try1")